# Step 1: Initialize environment

In [1]:
!rm -rf /content/sample_data
!nvidia-smi

Mon Mar  1 17:00:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch
!pip install -U albumentations
!pip install ttach
!pip install pytorch_toolbelt

In [3]:
!rm -rf /content/suichang_round1_train_210120
!rm -rf /content/suichang_round1_train_210120.zip
!cp -r /content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/suichang_round1_train_210120.zip ./
!unzip -q suichang_round1_train_210120.zip
!rm -rf /content/suichang_round1_train_210120.zip

# Step 2: Data Augmentation

In [4]:
import albumentations as A
import os.path as osp
import numpy as np
import pandas as pd
import pathlib, sys, os, random, time, glob
import numba, cv2, gc
from tqdm import tqdm
import torch.utils.data as D
import torchvision
from torchvision import transforms as T
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm_notebook

import shutil
from shutil import copyfile
from sklearn.model_selection import train_test_split

In [5]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/val

## train_test_split

In [7]:
def MakeData(PATH):
    df = pd.DataFrame()
    X = []
    y = []
    for i in (list({i.split('.')[0] for i in os.listdir(PATH)})):
        i = i.split('.')[0]
        X.append(PATH + '{}.tif'.format(i))
        y.append(PATH + '{}.png'.format(i))
        
    df['X']  =  X
    df['y']  =  y 
    return df

data_path = "/content/suichang_round1_train_210120/"
data = MakeData(data_path)

train_path = 'dataset/train/'
val_path = 'dataset/val/'

train_data, val_data = train_test_split(data, test_size=0.2, random_state=666)
train_data =data

print(len(train_data['X']),len(val_data['X']))
print(len(train_data['y']),len(val_data['y']))

16017 3204
16017 3204


In [8]:
for i in tqdm_notebook(range(0, len(train_data['X']))):
    shutil.copy(train_data['X'].iloc[i], train_path)
    shutil.copy(train_data['y'].iloc[i], train_path)
print(len(os.listdir(train_path))/2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.



16017.0


In [9]:
for i in tqdm_notebook(range(0, len(val_data['X']))):
    shutil.copy(val_data['X'].iloc[i], val_path)
    shutil.copy(val_data['y'].iloc[i], val_path)
print(len(os.listdir(val_path))/2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.



3204.0


## Data aug

In [10]:
IMAGE_SIZE =256
class DatasetAug(D.Dataset):
    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform

        self.len = len(paths)
        self.as_tensor = T.Compose([
            T.ToPILImage(),
            T.Resize(IMAGE_SIZE),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ])
    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

    # get data operation
    def __getitem__(self, index):
        img = cv2.imread(self.paths[index])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.paths[index].replace('.tif', '.png'),cv2.IMREAD_UNCHANGED) #- 1
        #mask = cv2.resize(mask, (IMAGE_SIZE, IMAGE_SIZE))
        transformed = self.transform(image=img, mask=mask)
        image = transformed["image"]
        mask = transformed["mask"]
        return image, mask

In [11]:
train_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        # A.Transpose(p=0.5),
        # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        # A.RandomSizedCrop([96, 192],256,256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        #A.RandomSizedCrop([128, 256], 256, 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [12]:
dataset_path = './'
train_dataset = DatasetAug(glob.glob(osp.join(dataset_path, 'dataset/train', '*.tif')),train_transform)
print(len(train_dataset))

16017


In [ ]:
dataset_path = './'
val_dataset = DatasetAug(glob.glob(osp.join(dataset_path, 'dataset/val', '*.tif')),val_transform)
print(len(val_dataset))

3204


## visualize augmentations

In [ ]:
import copy
import matplotlib.pyplot as plt
def visualize_augmentations(dataset, idx=0, samples=8):
    dataset = copy.deepcopy(dataset)
    transform =train_transform
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    figure, ax = plt.subplots(nrows=samples, ncols=2, figsize=(10, 24))
    for i in range(samples):
        image, mask = dataset[idx]
        ax[i, 0].imshow(image)
        ax[i, 1].imshow(mask) #, cmap='gray')
        ax[i, 0].set_title("Augmented image")
        ax[i, 1].set_title("Augmented mask")
        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()
    plt.tight_layout()
    plt.show()
random.seed(42)
visualize_augmentations(train_dataset, idx=233, samples=8)

In [17]:
!rm -rf dataset/train_aug
!rm -rf dataset/val_aug
!mkdir dataset/train_aug
!mkdir dataset/val_aug

In [18]:
import PIL
from PIL import Image, ImageChops, ImageEnhance
def save_augmentations(dataset, path, transform, idx=0, samples=4):
    dataset = copy.deepcopy(dataset)
    transform = transform
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    for i in range(samples):
        image, mask = dataset[idx]
        image=Image.fromarray(image)
        mask=Image.fromarray(mask)
        image.save(path+'{}_{}.tif'.format(i,idx))
        mask.save(path+'{}_{}.png'.format(i,idx))

In [19]:
for id in tqdm_notebook(range(len(train_dataset))):
    save_augmentations(train_dataset, 'dataset/train_aug/', train_transform, idx=id, samples=4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [20]:
!rm -rf /content/suichang_round1_train_210120

In [ ]:
for id in tqdm_notebook(range(len(val_dataset))):
    save_augmentations(val_dataset, 'dataset/val_aug/', val_transform, idx=id, samples=6)

In [ ]:
image_file_number=glob.glob('dataset/train_aug/*tif')
mask_file_number=glob.glob('dataset/train_aug/*png')
print(len(image_file_number),len(mask_file_number))

val_image_file_number=glob.glob('dataset/val_aug/*tif')
val_mask_file_number=glob.glob('dataset/val_aug/*png')
print(len(val_image_file_number),len(val_mask_file_number))

76878 76878
19224 19224


# Step 3: Model Training

In [21]:
import os.path as osp
import numpy as np
import pandas as pd
import pathlib, sys, os, random, time
import numba, cv2, gc
from tqdm import tqdm

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import albumentations as A
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
torch.backends.cudnn.enabled = True

import torchvision
from torchvision import transforms as T
import glob

from pytorch_toolbelt import losses as L
import pytest
import torch
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.losses._functional as F
from segmentation_models_pytorch.losses import DiceLoss, JaccardLoss, SoftBCEWithLogitsLoss, SoftCrossEntropyLoss
from segmentation_models_pytorch.losses import DiceLoss,FocalLoss,SoftCrossEntropyLoss


from pytorch_toolbelt import losses as L
from albumentations.pytorch import ToTensorV2
from segmentation_models_pytorch.losses import DiceLoss,FocalLoss,SoftCrossEntropyLoss,LovaszLoss

## Paramers

In [22]:
EPOCHES = 60
BATCH_SIZE = 16
IMAGE_SIZE = 256

# cuDNN使用的非确定性算法就会自动寻找最适合当前配置的高效算法，来达到优化运行效率的问题
torch.backends.cudnn.enabled = True
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Pipline

In [23]:
transform = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    #ToTensorV2(),
])

# transform = A.Compose(
#     [
#         A.Resize(256, 256),
#         A.VerticalFlip(p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.Transpose(p=0.5),
#         A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
#         A.RandomSizedCrop([96, 192],256,256),
#         A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#         ToTensorV2(),
#     ]
# )
class Trainset(D.Dataset):

    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform
       

        self.len = len(paths)
        self.as_tensor = T.Compose([
            T.ToPILImage(),
            T.Resize(IMAGE_SIZE),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ])

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

    def __getitem__(self, index):
        img = cv2.imread(self.paths[index], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        mask = cv2.imread(self.paths[index].replace('.tif', '.png'))  - 1 
        mask = cv2.resize(mask, (IMAGE_SIZE, IMAGE_SIZE))
        augments = self.transform(image=img, mask=mask)
        return self.as_tensor(augments['image']), augments['mask'][:, :, 0].astype(np.int64)       

In [25]:
val_idx, train_idx = [], []

dataset_path ='dataset'
train_dataset = Trainset(glob.glob(osp.join(dataset_path, 'train_aug', '*.tif')),transform)
print("---train_dataset Size---")
print(len(train_dataset))

val_dataset = Trainset(glob.glob(osp.join(dataset_path, 'val', '*.tif')),transform)
print("---val_dataset Size---")
print(len(val_dataset))

# dataset_path ='./'
# train_dataset = Trainset(glob.glob(osp.join(dataset_path, 'suichang_round1_train_210120', '*.tif')),transform)
# print("---train_dataset Size---")
# print(len(train_dataset))


for i in range(len(train_dataset)):
    train_idx.append(i) 
    #if i % 4 == 0:
for i in range(len(val_dataset)):
    val_idx.append(i)
           
print("---idx Size---")
print(len(train_idx),len(val_idx))

train_ds = D.Subset(train_dataset, train_idx)
valid_ds = D.Subset(val_dataset, val_idx)

# define training and validation data loaders
loader = D.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
vloader = D.DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
print("---DataLoader Size---")
print(int(len(loader)), int(len(vloader)))

---train_dataset Size---
64068
---val_dataset Size---
3204
---idx Size---
64068 3204
---DataLoader Size---
4005 201


In [26]:
def get_iou(pred, mask, c=10):
    iou_result = []
    for idx in range(c):
        p = (mask == idx).int().reshape(-1)
        t = (pred == idx).int().reshape(-1)

        uion = p.sum() + t.sum()
        overlap = (p * t).sum()

        iou = 2 * overlap / (uion + 0.001)
        iou_result.append(iou.abs().data.cpu().numpy())
    return np.stack(iou_result)

def validation(model, loader, loss_fn):
    model.eval()
    with torch.no_grad():
        val_iou = []
        for image, target in loader:
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)
            output = output.argmax(1)
            iou = get_iou(output, target)
            val_iou.append(iou)
    return val_iou

## Model

In [ ]:
# repo: https://github.com/qubvel/segmentation_models.pytorch
# doc: https://smp.readthedocs.io/en/latest/

header = r'''Epoch |  Loss |  Score | Time(min)'''
raw_line = '{:8d}' + '\u2502{:8f}' * 2 + '\u2502{:8f}'
class_name = ['farm', 'land', 'forest', 'grass', 'road', 'urban_area','countryside', 'industrial_land', 'construction', 'water', 'bareland']

model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights='imagenet', in_channels=3, classes=10)
#model.load_state_dict(torch.load("/content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/model/upp_efb6_aug2_12.pth"))
model.train()
model.to(DEVICE)

# optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0005)

# loss 
# DiceLoss, JaccardLoss, SoftBCEWithLogitsLoss, SoftCrossEntropyLoss
DiceLoss_fn = DiceLoss(mode='multiclass')
SoftCrossEntropy_fn=SoftCrossEntropyLoss(smooth_factor=0.1)
loss_fn = L.JointLoss(first=DiceLoss_fn, second=SoftCrossEntropy_fn,first_weight=0.5, second_weight=0.5).to(DEVICE)


best_iou = 0

for epoch in (range(1, EPOCHES + 1)):   
    losses = []
    start_time = time.time()

    for image, target in tqdm(loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    viou = validation(model, vloader, loss_fn)

    print(header)
    print(raw_line.format(epoch, np.array(losses).mean(), np.mean(viou),
                            (time.time() - start_time) / 60 ** 1))
    print('\n')    
    print('  '.join(class_name))
    print('\t'.join(np.stack(viou).mean(0).round(3).astype(str)))
   
    if best_iou < np.stack(viou).mean(0).mean():
        best_iou = np.stack(viou).mean(0).mean()
        model_path = '/content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/model/'
        torch.save(model.state_dict(), model_path + 'upp_efb6_{}.pth'.format(epoch))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth



Epoch |  Loss |  Score | Time(min)
       1│0.718140│0.394419│30.403612


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.76	0.947	0.0	0.335	0.121	0.674	0.263	0.084	0.76	0.0



Epoch |  Loss |  Score | Time(min)
       2│0.669429│0.427197│29.118430


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.778	0.951	0.0	0.345	0.221	0.688	0.295	0.215	0.778	0.0



Epoch |  Loss |  Score | Time(min)
       3│0.646380│0.465752│29.137220


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.795	0.954	0.107	0.405	0.226	0.714	0.332	0.326	0.799	0.0



Epoch |  Loss |  Score | Time(min)
       4│0.629225│0.495121│29.173315


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.81	0.956	0.195	0.436	0.269	0.733	0.359	0.385	0.809	0.0


...


Epoch |  Loss |  Score | Time(min)
      14│0.556902│0.616481│29.460291


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.847	0.965	0.512	0.546	0.432	0.785	0.454	0.562	0.851	0.21



Epoch |  Loss |  Score | Time(min)
      15│0.553034│0.626365│29.401593


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.853	0.966	0.525	0.573	0.445	0.791	0.461	0.568	0.858	0.223



Epoch |  Loss |  Score | Time(min)
      16│0.550045│0.630426│29.391946


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.854	0.967	0.529	0.574	0.447	0.791	0.463	0.59	0.862	0.228



Epoch |  Loss |  Score | Time(min)
      17│0.547462│0.623170│29.446246


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.843	0.964	0.499	0.568	0.446	0.788	0.466	0.582	0.853	0.223



Epoch |  Loss |  Score | Time(min)
      18│0.544063│0.634511│29.518830


farm  land  forest  grass  road  urban_area  countryside  industrial_land  construction  water  bareland
0.858	0.967	0.546	0.575	0.449	0.792	0.471	0.591	0.862	0.235
